In [1]:
from library_buatan.holdout_workflow_comvis import Holdout_Workflow_CV
from library_buatan.holdout_workflow_comvisV2 import Holdout_Workflow_CV2
# from mobilenetv2cbam.mobilenetv2cbam_hilmi import MobileNetV2_CBAM_hilmi
# from mobilenetv2.mobilenetv2 import MobileNetV2
# from basic_workflow_comvis import Pov2ImageDataset_V4

import torch
from torch.utils.data import DataLoader
from torch import nn
from torch import optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from torchvision import models
from torchvision.models.mobilenetv3 import _mobilenet_v3_conf

import numpy as np

import pandas as pd

from PIL import Image

import matplotlib.pyplot as plt

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# target_names = ['Apple___Apple_scab','Apple___Black_rot','Apple___Cedar_apple_rust','Apple___healthy']

target_names = {0:'Apple_scab',
                1:'Black_rot',
                2:'Cedar_apple_rust',
                3:'healthy'}

workflow = Holdout_Workflow_CV2(target_names, device)
inverted_residual_setting, last_channel = _mobilenet_v3_conf('mobilenet_v3_small')

Using device: cuda:0


In [4]:

mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])

transform_train = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    # transforms.RandomAffine(degrees=(-15, 15), shear=(-15, 15)),

#   transforms.Lambda(lambda img: imgaug_transform.augment_image(np.array(img))),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])

transform_test = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean,std),
])

train_dataset = datasets.ImageFolder(root='apple_leaf_dataset/backbone/train_backbone',
                                           transform=transform_train,)
                                        #    target_transform=transforms.Lambda(lambda label: torch.tensor(target_names[label])) )
valid_dataset = datasets.ImageFolder(root='apple_leaf_dataset/backbone/valid_backbone',
                                           transform=transform_test,)
                                        #    target_transform=transforms.Lambda(lambda label: torch.tensor(target_names[label])) )
test_dataset = datasets.ImageFolder(root='apple_leaf_dataset/test',
                                           transform=transform_test,)

test_dataset2 = datasets.ImageFolder(root='apple_leaf_dataset/rcnn/valid_rcnn',
                                           transform=transform_test,)

### Uji Coba

In [5]:
# name_file = f"kmu_fed_kfold{n_splits}.{fi}.{epochs}_mobilenetv2_v0.3"
name_model = "mobilenetv3small"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/mobilenetv3small"

# path_dir = "../resources"
name_data = "apple_leaf_disease"
# path = path_dir+"/isdds_skeleton/90c_v2"
# dataset_list = [path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/latih_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/valid_isdds90c_irhv2.csv",
#                 path_dir+"/isdds_irhv2_nonpersonsplitv2_holdout/uji_isdds90c_irhv2.csv",]
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

criterion = nn.CrossEntropyLoss()

optim_params = {}
optimizer = optim.Adam
# scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5)

patience = 15#30
# counter = 0

# n_splits=5
epochs=1
batch_size= 16#8#64#32

versi = "Adam_default_outchan1280_16batch" #X, coba lr0.001

In [6]:
# workflow.run_workflow(
#     versi=versi, model=model, str_to_num=str_to_num,
#     # init_model=init_mobilenetv2(),
#     # dataset_const=dataset,
#     batch_size=batch_size,
#     # n_splits=n_splits,
#     epochs=epochs,
#     patience=patience,
#     # class_names=class_names,
#     path='',
#     name_model=name_model,
#     name_data=name_data,
#     save_path0=save_path,
#     criterion_const=criterion,
#     optim_const=optimizer, optim_params=optim_params,
#     transform_train=transform_train, transform_test=transform_test,
#     dataset_list=None, #dataset_list
#     dataset_list2=dataset_list2

# )

workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    # pilihan='y',
)


	Epochs 1
		Best -> epoch: 1
		Training Loss: 0.426
			Training Accuracy: 95.259%
			Training Precision: 95.326%
			Training F1: 95.255%
			Training Recall: 95.259%
		Validation loss: 37.924
			Validation Accuracy: 25.989%
			Validation Precision: 6.754%
			Validation F1: 10.722%
			Validation Recall: 25.989%
		ROC AUC validation: 0.5203330779087134
	Waktu komputasi training: 33.41113209724426 detik
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       429
           1       0.00      0.00      0.00       422
           2       0.00      0.00      0.00       365
           3       0.26      1.00      0.41       427

    accuracy                           0.26      1643
   macro avg       0.06      0.25      0.10      1643
weighted avg       0.07      0.26      0.11      1643

	Finished Training: apple_leaf_disease_1_mobilenetv3small_Adam_default_outchan1280_16batch


		Test loss: 0.368
			Test Accuracy: 11.111%
			Test Precision: 1.235

### Apple Leaf Disease

#### Default

In [ ]:
# name_file = f"kmu_fed_kfold{n_splits}.{fi}.{epochs}_mobilenetv2_v0.3"
name_model = "mobilenetv3small"
model = models.MobileNetV3(inverted_residual_setting=inverted_residual_setting,
                    last_channel=last_channel,
                    num_classes=4) #MobileNetV2_CBAM_hilmi(10)
str_to_num=False
save_path = f"hasil/mobilenetv3small"

name_data = "apple_leaf_disease"
dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset
]

criterion = nn.CrossEntropyLoss()

optim_params = {}
optimizer = optim.Adam

patience = 15
epochs=20
batch_size= 16 

versi = "Adam_default_outchan1280_16batch"

In [ ]:
workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    pilihan='y',
)


Folder tersebut sudah ada, yakin tetap melanjutkan fungsi run_workflow()?(y/n)
status use_train:  True
fungsi tetap dijalankan
	Epochs 20
		Best -> epoch: 16
		Training Loss: 0.050
			Training Accuracy: 98.390%
			Training Precision: 98.390%
			Training F1: 98.390%
			Training Recall: 98.390%
		Validation loss: 0.013
			Validation Accuracy: 99.757%
			Validation Precision: 99.757%
			Validation F1: 99.757%
			Validation Recall: 99.757%
		ROC AUC validation: 0.9999600235700408
	Waktu komputasi training: 654.084587097168 detik
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       429
           1       1.00      1.00      1.00       422
           2       1.00      0.99      1.00       365
           3       1.00      1.00      1.00       427

    accuracy                           1.00      1643
   macro avg       1.00      1.00      1.00      1643
weighted avg       1.00      1.00      1.00      1643

	Finished Training: apple_leaf_dis

In [6]:
model.load_state_dict( torch.load("hasil/mobilenetv3small/apple_leaf_disease_20_mobilenetv3small_Adam_default_outchan1280_16batch/apple_leaf_disease_20_mobilenetv3small_Adam_default_outchan1280_16batch.pth") )

dataset_list2 = [
    train_dataset,
    valid_dataset,
    test_dataset2
]

workflow.run_demo(
    versi=versi, model1=model, str_to_num=str_to_num,
    # init_model=init_mobilenetv2(),
    # dataset_const=dataset,
    batch_size=batch_size,
    # n_splits=n_splits,
    epochs=epochs,
    patience=patience,
    # class_names=class_names,
    path='',
    name_model=name_model,
    name_data=name_data,
    save_path0=save_path,
    criterion_const=criterion,
    optim_const=optimizer, optim_params=optim_params,
    transform_train=transform_train, transform_test=transform_test,
    dataset_list=None, #dataset_list
    dataset_list2=dataset_list2,
    
    pilihan='y',
    use_train=False,
)


Folder tersebut sudah ada, yakin tetap melanjutkan fungsi run_workflow()?(y/n)
status use_train:  False
fungsi tetap dijalankan
	Epochs bukan train
		Best -> epoch: bukan train
		Training Loss: 0.007
			Training Accuracy: 99.778%
			Training Precision: 99.779%
			Training F1: 99.779%
			Training Recall: 99.778%
		Validation loss: 0.013
			Validation Accuracy: 99.757%
			Validation Precision: 99.757%
			Validation F1: 99.757%
			Validation Recall: 99.757%
		ROC AUC validation: 0.9999600235700408
	Waktu komputasi training: 20.312864780426025 detik
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       429
           1       1.00      1.00      1.00       422
           2       1.00      0.99      1.00       365
           3       1.00      1.00      1.00       427

    accuracy                           1.00      1643
   macro avg       1.00      1.00      1.00      1643
weighted avg       1.00      1.00      1.00      1643

	Finished Tra